# LAPD 'Part I' crimes database: 2010-present

### Import data tools

In [1]:
import pandas as pd
import geopandas as gpd
import pyarrow
import matplotlib
import matplotlib.pyplot as plt
import geojson
import json
import jenkspy
import numpy as np
from earthpy import clip as cl
from altair import datum
import weightedcalcs as wc
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

### Read crimes CSV downloaded from LA City data portal

In [2]:
# data exported from '00-lapd-crimes-processing.ipynb'
crimes = pd.read_feather('/Users/mhustiles/data/data/LA/crimes.feather')

---

### Understanding LAPD's modus operandi codes

In [3]:
mocrimes = crimes.dropna(subset=['modus_operandi_code'])

In [4]:
mocodes = pd.read_csv('mo_codes.csv')
mocodes.head()

,mo_code,mo_code_description
0,0100,Suspect Impersonate
1,0101,Aid victim
2,0102,Blind
3,0103,Crippled
4,0104,Customer


### Isolating cases involving specific 'MO' tags (ie homeless, , gang, etc)

In [5]:
# For example...
mocodes_homeless = mocodes[mocodes['mo_code_description'].str.lower().str.contains('gang')]
mocodes_homeless.head()

,mo_code,mo_code_description
114,0371,Gang affiliation questions asked/made gang sta...
117,0374,Gang signs/threw gang signs using hands
270,0906,Gangs
312,0946,Gang Feud
410,1270,Victim was gang member


---

### Parsing codes

In [6]:
df = crimes[['record_id', 'modus_operandi_code']]

In [7]:
df['modus_operandi_code'] = df['modus_operandi_code'].dropna().apply(lambda x: x.split(' '))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
final = df['modus_operandi_code'].apply(pd.Series)\
.merge(df, left_index = True, right_index = True)\
.drop('modus_operandi_code', axis = 1)\
.melt(id_vars = ['record_id'], value_name = "modus_operandi_code", var_name = 'modoporder')\
.dropna(subset=['modus_operandi_code'])\
.sort_values('record_id')

In [9]:
mo_counts = final.groupby(['modus_operandi_code']).agg('size')\
.reset_index(name='count').sort_values(by='count', ascending=False)

In [10]:
merged_mo_counts = pd.merge(mo_counts, mocodes, 
                     left_on = 'modus_operandi_code', 
                     right_on = 'mo_code', 
                     how='left')

In [11]:
merged_mo_counts.head()

,modus_operandi_code,count,mo_code,mo_code_description
0,0344,679188,0344,Removes vict property
1,0416,81365,0416,Hit-Hit w/ weapon
2,1609,80708,1609,Smashed
3,1402,78798,1402,Evidence Booked (any crime)
4,1300,73373,1300,Vehicle involved


---